In [31]:
import tmdbsimple as tmdb
import os, time, json
import pandas as pd
import tmdbsimple as tmdb
from tqdm import tqdm_notebook

In [32]:
def get_movie_with_rating(movie_id):
    """Apapted from source= https://github.com/celiao/tmdbsimple"""
    # get movie object for movie id
    movie = tmdb.Movies(movie_id)

    # save info and releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # loop through countries in releases dictionary
    for c in releases['countries']:
        # if country = US, create a certification key in info with the certification from releases
        if c['iso_3166_1'] == 'US':

            info['certification']=c['certification']
    return info

In [33]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a existing json file. 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename,'r+') as file:
        # load existing data into a dict
        file_data = json.load(file)
        # choose to extend or append new data based on data type
        if (type(new_data)==list) & (type(file_data)==list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # sets file's current position at offset
        file.seek(0)
        # convert file to json
        json.dump(file_data, file)

In [34]:
# load API key from json fie
with open('/Users/x471074/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [35]:
tmdb.API_KEY =  login['api-key']

In [36]:
# testing get_movie_rating function
get_movie_with_rating("tt0848228")

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 128.625,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [37]:
get_movie_with_rating("tt0332280")

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 56.096,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/mevhneWSqbjU22D1MXNd4H9x0r0.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [38]:
# specify folder to load data
FOLDER = 'Data/'
# create folder
os.makedirs(FOLDER, exist_ok=True)
# display files in folder
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [39]:
# load in the datafram from project part 1
basics = pd.read_csv('Data/title_basics.csv.gz')

In [40]:
YEARS_TO_GET = [2000, 2001]
errors = []

for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    # defining JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # if it does not exists: create it
    if file_exists == False:
    # save an empty dict with just 'imdb_id' to the new json file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        
    # saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie_ids to list
    movie_ids = df['tconst'].copy()
    # load existsing data from json into dataframe 
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    # get movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):

        try:
            # retrieve the data for the movie id
            temp = get_movie_with_rating(movie_id)
            # append/extend results to existing file
            write_json(temp,JSON_FILE)
            # 20 ms sleep to avoid overwhelming the server
            time.sleep(0.02)

        # if there's errors, append to error list
        except Exception as e:
            errors.append([movie_id,e])
    
    # read json file
    final_year_df = pd.read_json(JSON_FILE)
    # save file to csv.gz
    final_year_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz',compression='gzip',index=False)

# print errors
print(f'-Total errors: {len(errors)}')

C:\Users\x471074\AppData\Local\Temp\ipykernel_7720\727502443.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\x471074\AppData\Local\Temp\ipykernel_7720\727502443.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie_id in tqdm_notebook(movie_ids_to_get,


Movies from 2000:   0%|          | 0/1462 [00:00<?, ?it/s]

C:\Users\x471074\AppData\Local\Temp\ipykernel_7720\727502443.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie_id in tqdm_notebook(movie_ids_to_get,


Movies from 2001:   0%|          | 0/1585 [00:00<?, ?it/s]

-Total errors: 426
